In [5]:
import math
import random

In [335]:
# Lmax: Retransmit limit
# TXOP: in ms
# Imax: total slots in worst case
def simulation(sta_amount, RA_RU, Lmax, TXOP, ocw_min, ocw_max, mode='general'):
    counter_delay_slot=0
    success_sta=0
    # setting ocw_n [ocw_min, 1st_retry_ocw....]
    ocw = [0 for _ in range(0, Lmax)]
    ocw[0] = ocw_min
    for n in range (1, Lmax):
        temp_ocw = ocw[n-1] * 2 + 1
        if temp_ocw <= ocw_max:
            ocw[n] = temp_ocw
        else:
            ocw[n] = ocw[n-1]
    # Imax
    Imax = 0
    for n in range(0, Lmax):
        Imax += math.ceil(ocw[n] / RA_RU)
    # ------------------------------------ STA setting ------------------------------------------
    # set each STA's OBO counter
    obo_counter = [random.randint(0, ocw_min) for _ in range(0, sta_amount)]
    # set STA's transmit attempt time
    retrans_counter = [0 for _ in range(0, sta_amount)]
#     # selected RA-RU index for each station
#     sta_selected_R = [None for _ in range(0, sta_amount)]
    # ------------------------------------ STA setting ------------------------------------------
    # number of STAs on RA-RU
    sta_on_R = [[[] for _ in range(0,RA_RU)] for _ in range(0,Imax)]
    
    success_info = []
    # No.i slot    
    for i in range(0, Imax):
        # No.x STA
        for x in range (0,sta_amount):
            # decrease OBO counter by RA-RU number
            obo_counter[x] -= RA_RU
            # if counter <=0 select R
            if (obo_counter[x] <= 0 and retrans_counter[x] < 5):
                selected_R_index = random.randint(0, RA_RU-1)
                sta_on_R[i][selected_R_index].append(x)
        # check the balls in R
        for r in range(0, len(sta_on_R[i])):
            # success STA
            if (len(sta_on_R[i][r]) == 1):
                success_sta_index = sta_on_R[i][r][0]
                # mark 99999 = won't trasnmit again
                obo_counter[success_sta_index] = 99999
                # record
                counter_delay_slot += (i+1)
                success_sta += 1
            else:
                for fail_idx in sta_on_R[i][r]:
                    retrans_counter[fail_idx] += 1
                    if (retrans_counter[fail_idx] < 5):
                        obo_counter[fail_idx] = random.randint(0, ocw[retrans_counter[fail_idx]])
#     print(counter_delay_slot)
#     print(success_sta)
    success_probability = success_sta / sta_amount
    access_delay = 0
    if success_sta > 0:
        access_delay = (TXOP * counter_delay_slot) / success_sta
    
    through_put = success_sta / Imax
    utilization = success_sta / (Imax * RA_RU)
    
    return success_probability, access_delay, through_put, utilization
    


In [339]:
sample = 1000

m_list_simu = [10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200]
Ps_list_simu = [0 for i in range(20)]
Da_list_simu = [0 for i in range(20)]
T_list_simu = [0 for i in range(20)]
U_list_simu = [0 for i in range(20)]

R=74

for idx, m in enumerate(m_list_simu):
    success_probability_total = 0;
    access_delay_total = 0;
    through_put_total = 0;
    utilization_total = 0;
    for i in range(sample):
        success_probability, access_delay, through_put, utilization = simulation(sta_amount=m, RA_RU=R, Lmax=5, TXOP=5.673, ocw_min=7, ocw_max=31, mode='general')
        success_probability_total += success_probability
        access_delay_total += access_delay
        through_put_total += through_put
        utilization_total += utilization

    Ps = success_probability_total / sample
    Da = access_delay_total / sample
    T = through_put_total / sample
    U = utilization_total / sample
    
    Ps_list_simu[idx] = Ps
    Da_list_simu[idx] = Da
    T_list_simu[idx] = T
    U_list_simu[idx] = U

print(Ps_list_simu)
print(Da_list_simu)
print(T_list_simu)
print(U_list_simu)

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.999942857142857, 0.9994750000000002, 0.998755555555555, 0.9947500000000026, 0.9850545454545507, 0.9636083333333384, 0.927792307692297, 0.8702499999999999, 0.802246666666666, 0.7279937500000004, 0.6463117647058834, 0.5693388888888891, 0.49451052631578957, 0.43104999999999993]
[6.342981299999955, 7.035087300000035, 7.7733337000000144, 8.448373425000051, 9.256293720000016, 10.089808699999992, 10.982825504621841, 11.934976735880584, 12.963572132419694, 13.976710461206618, 15.03294053842881, 15.973086469713307, 16.82770849270516, 17.43948811344247, 17.87586546784776, 18.139953366963084, 18.297012176323875, 18.40746388290733, 18.373619490976324, 18.35358546940912]
[2.0, 4.0, 6.0, 8.0, 10.0, 12.0, 13.9992, 15.991600000000004, 17.977600000000013, 19.894999999999996, 21.671199999999995, 23.1266, 24.122600000000002, 24.366999999999997, 24.067400000000006, 23.295800000000014, 21.974599999999988, 20.49619999999999, 18.791399999999985, 17.241999999999994]
[0.0270270